In [5]:
import numpy as np
import pandas as pd
from Blastn import *

In [6]:
def write_fasta(items, fname='/Users/rfor10/Downloads/temp.fa'):
    if isinstance(items, list):
        items = {idx: i for idx, i in enumerate(items)}
    if not isinstance(items, dict):
        return None
    
    with open(fname, 'w') as f:
        for k, v in items.items():
            f.write('>'+str(k)+'\n'+v+'\n')
def check_homology_readouts(hit_list):
    bad_ids = set()
    min_length = 1000
    for h in hit_list_readouts:
        fields = str(h).split('\t')
        h_id = float(fields[2].split(':')[0])
        subject = float(fields[4].split(':')[0])
        length = int(fields[5].split(':')[1].strip(' '))
        identity = float(fields[6].split(':')[1].strip(' ').strip('%'))
        if (length < min_length) and h_id != subject:
            min_length = length
        if length>11 and (identity==100) and (h_id != subject):
            bad_ids.add(h_id)
#         if length <=11:
#             print('hi')
#             break
    return list(bad_ids), min_length

In [9]:
blastpth = '/Users/robertf/Downloads/ncbi-blast-2.6.0+/bin/makeblastdb'

nseqs = 100000

In [229]:
pos_readouts = [''.join(np.random.choice(['A', 'T', 'G'], size=20, p=[.25, .25, .5])) for i in range(nseqs)]
pos_readouts = [i for i in pos_readouts if (i.count('G')>0.4*20 and i.count('G')<0.5*20)]
pos_readouts = [i for i in pos_readouts if 'GGGG' not in i]

In [230]:
print('Number initial readouts:', len(pos_readouts))
pos_readouts = {float(idx):v for idx, v in enumerate(pos_readouts)}
write_fasta(pos_readouts)

Number initial readouts: 11019


In [4]:
blast = Blastn('/Users/rfor10/Downloads/gencode.v26_and_mouse.vM13.transcripts.fa', makeblastdb_exec=blastpth)

Error, no data received from standard output
/bin/sh: /Users/rfor10/Google Drive/wollman/tools/ncbi-blast-2.6.0+/bin/makeblastdb: No such file or directory



In [10]:
high_expressed = Blastn('/Users/robertf/Downloads/MERFISH_Examples2/Homo_sapiens.GRCh38.ncrna.fa', makeblastdb_exec=blastpth)

In [29]:
high_list = high_expressed(query_path='/Users/robertf/Google\ Drive/hypefish/ipython/ca_gene_targets/disulfide_readouts.fasta', task='blastn', evalue=.001000000000000000)
bad_ids = set()
for h in high_list:
    fields = str(h).split('\t')
    h_id = float(fields[2].split(':')[0])
    length = int(fields[5].split(':')[1].strip(' '))
    identity = float(fields[6].split(':')[1].strip(' ').strip('%'))
    if length>11 and (identity==100):
        bad_ids.add(h_id)
bad_ids = list(bad_ids)

In [231]:
hit_list = blast(query_path='/Users/rfor10/Downloads/temp.fa', task='blastn-short', evalue=500, best_query_hit=True)
bad_ids = set()
for h in hit_list:
    fields = str(h).split('\t')
    h_id = float(fields[2].split(':')[0])
    length = int(fields[5].split(':')[1].strip(' '))
    identity = float(fields[6].split(':')[1].strip(' ').strip('%'))
    if length>11 and (identity==100):
        bad_ids.add(h_id)
bad_ids = list(bad_ids)
[pos_readouts.pop(i) for i in bad_ids]
write_fasta(pos_readouts)

In [237]:
readout_blast = Blastn('/Users/rfor10/Downloads/temp.fa', makeblastdb_exec = blastpth)

In [238]:
hit_list_readouts = readout_blast(query_path='/Users/rfor10/Downloads/temp.fa', task='blastn-short', evalue=.1, best_query_hit=False)
bad_ids, minl = check_homology_readouts(hit_list_readouts)
print('Minimum length at this evalue:', minl)
pos_readouts2 = pos_readouts
[pos_readouts2.pop(i) for i in bad_ids]

Minimum length at this evalue: 10


['TGGTGAGAGTTAGGGTTAGA',
 'AGGGATTGAGTGTGATATGG',
 'AGTGTTGAAGTGGATGGAAG',
 'GTGGTAATTTTAGGTGGTGG',
 'TTGGATTGAATGGGAGGAAG',
 'GTATGATGGTAGGTTGGGTA',
 'TAGTGAAAGAGTGGAAGGTG',
 'AATGGGTTAGTGGGTTGATG',
 'TGGGATATTGAAGGTGAGAG',
 'TTTGGTGGGTATAAGGGAGA',
 'TAGTAGGAGATGTAAGTGGG',
 'GGGTATGGTATAAGAGGTAG',
 'GTGTAAGGGATGTATAGGAG',
 'GATGAGTATGTGAGGTAGTG',
 'TGTTTAGTATGGTGAGGTGG',
 'TGGGAATGATGGTAGTGTGT',
 'ATGGTGATAGGAGGAGAATG',
 'GGAATAGAGGAGTAAGAGGT',
 'TGATGTGGTGTGGTAATGAG',
 'TGAGGATAGAGGAGTAAAGG',
 'AGGAAAGTGAGAGGTTGGAA',
 'GGATGTATAGGAGGGTGATT',
 'GGTGGGTATAAGTTGTAGAG',
 'TAAGTGGATGGTGTGTGTGT',
 'GTGAGAGTTAGGTTGAGAAG',
 'AGATGTGGTGTGGATGTGAT',
 'AAGGGTGATAGGAGGAAGAA',
 'GGAGGGTTTATAAGAGGTAG',
 'TAGGGTGGAAGTTAAGGAGA',
 'GATGAGAGGTTGGAGATGTA',
 'GGGTGATGTTGAAGTGGAAT',
 'GGTGATAGGATTGGTAGTAG',
 'GGTGGTAAGATTGAGTGTGA',
 'TGTTGTGTGAGGTAGTGAGT',
 'GTATGGTGGTAGTTGGTATG',
 'AGGTGGGAGTTAAAGAGAAG',
 'GAAATAGAGGTGGGTTGGTA',
 'GGAGAGATGGTGTGTGTTTA',
 'GTGGGAATGATGGTGAATAG',
 'AAGAGTGAAAGAGTGAGGAG',


In [246]:
pop_list = []
for k, v in pos_readouts2.items():
    if 'GGG' in v:
        pop_list.append(k)
for k in pop_list:
    pos_readouts2.pop(k)
len(pos_readouts2)

171

In [247]:
write_fasta(pos_readouts2, fname='/Users/rfor10/Google Drive/hypefish/ipython/ca_gene_targets/new_readouts.fa')

In [99]:
good_ids = list(set(range(len(pos_readouts)))-bad_ids)

In [86]:
hit_hist_merprobes = blast(query_path='/Users/rfor10/Downloads/MERFISH_Examples2/readouts.fasta', task='blastn-short', best_query_hit=True)

In [92]:
for h in hit_hist_merprobes:
    fields = str(h).split('\t')
    h_id = int(fields[0].split(' ')[1])
    length = int(fields[5].split(':')[1].strip(' '))
    identity = float(fields[6].split(':')[1].strip(' ').strip('%'))
    if length>11 and (identity==100):
        print(h.q_id)
        print(h.s_id)
        print(h.length)

RS0015
ENSMUST00000171423.7|ENSMUSG00000024659.14|OTTMUSG00000035986.2|OTTMUST00000092344.2|Anxa1-002|Anxa1|362|processed_transcript|
16
RS0015
ENST00000608623.1|ENSG00000235437.7|OTTHUMG00000021699.2|OTTHUMT00000471676.1|LINC01278-004|LINC01278|5053|retained_intron|
16
RS0083
ENST00000541363.5|ENSG00000149948.13|OTTHUMG00000168936.2|OTTHUMT00000401662.1|HMGA2-011|HMGA2|8094|protein_coding|
16
RS0247
ENSMUST00000149514.7|ENSMUSG00000086316.8|OTTMUSG00000019378.4|OTTMUST00000046249.2|2210013O21Rik-003|2210013O21Rik|775|protein_coding|
17
RS0247
ENST00000576359.1|ENSG00000141542.10|OTTHUMG00000177806.1|OTTHUMT00000439010.1|RAB40B-003|RAB40B|2522|retained_intron|
16
RS0247
ENST00000269347.10|ENSG00000141542.10|OTTHUMG00000177806.1|OTTHUMT00000439009.1|RAB40B-002|RAB40B|2189|protein_coding|
16
RS0247
ENST00000571995.5|ENSG00000141542.10|OTTHUMG00000177806.1|OTTHUMT00000439007.1|RAB40B-001|RAB40B|3859|protein_coding|
16
RS0255
ENST00000533797.1|ENSG00000165323.15|OTTHUMG00000154468.4|OTTHUM